In [1]:
import matchzoo as mz
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/home/lss9/anaconda3/envs/matchzoo/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.
/home/lss9/anaconda3/envs/matchzoo/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
train_dataset = pd.read_csv('../../train_data/train_data_1_1', delimiter=',')
validation_dataset = pd.read_csv('../../train_data/validation_data_1_1', delimiter=',')

train_dataset = train_dataset.replace(np.nan, ' ', regex=True)
validation_dataset = validation_dataset.replace(np.nan, ' ', regex=True)

In [3]:
train_dataset = train_dataset.head(1000)
validation_dataset = validation_dataset.head(1000)

In [4]:
list_data1 = []
for i, row in train_dataset.iterrows():
    
    line1 = {'id_left': str(row['article_id']),
            'text_left':str(row['article_page_title'])+" "+str(row['article_meta_description']),
            'id_right':str(row['table_id']),
            'text_right':str(row['table_page_title'])+" "+str(row['table_page_summary']),
            'label':row['label']
           }
    
    list_data1.append(line1)

df1 = pd.DataFrame(list_data1)
train_pack = mz.pack(df1)

In [5]:
list_data2 = []
for i, row in validation_dataset.iterrows():
    
    line2 = {'id_left': str(row['article_id']),
            'text_left':str(row['article_page_title'])+" "+str(row['article_meta_description']),
            'id_right':str(row['table_id']),
            'text_right':str(row['table_page_title'])+" "+str(row['table_page_summary']),
            'label':row['label']
           }
    
    list_data2.append(line2)

df2 = pd.DataFrame(list_data2)
valid_pack = mz.pack(df2)

In [6]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankHingeLoss())
ranking_task.metrics = [
    mz.metrics.MeanAveragePrecision()
]

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)

In [8]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)
valid_pack_processed = preprocessor.transform(valid_pack)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 989/989 [00:00<00:00, 4612.67it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 996/996 [00:00<00:00, 1997.35it/s]
Building Vocabulary from a datapack.: 100%|██████████| 88353/88353 [00:00<00:00, 4207495.13it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 989/989 [00:00<00:00, 3097.36it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 996/996 [00:00<00:00, 1636.57it/s]
Processing text_right with transform: 100%|██████████| 996/996 [00:00<00:00, 78333.52it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval => StopRemoval: 100%|██████████| 988/988 [00:00<00:00, 5050.29it/s]
Processing text_right with chain_transform of Tokenize => Lo

In [9]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f582ccce790>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f582b271450>,
 'vocab_size': 10670,
 'embedding_input_dim': 10670,
 'input_shapes': [(10,), (100,)]}

In [10]:
model = mz.models.MVLSTM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = 300
model.params['lstm_units'] = 50
model.params['top_k'] = 20
model.params['mlp_num_layers'] = 2
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 5
model.params['mlp_activation_func'] = 'relu'
model.params['dropout_rate'] = 0.5
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             3201000     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 100)      140400      embedding[0][0]            

In [11]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [12]:
pred_x, pred_y = valid_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y), model_save_path='MVLSTM_title_main_passage', once_every=1)

In [13]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)

In [14]:
history = model.fit_generator(train_generator, epochs=5, callbacks=[evaluate])

Epoch 1/5
1/1 [==============================] - 2s 2s/step - loss: 1.0047
Validation: mean_average_precision(0.0): 0.5101214574898786
Epoch 2/5
1/1 [==============================] - 0s 66ms/step - loss: 1.0048
Validation: mean_average_precision(0.0): 0.5106275303643725
Epoch 3/5
1/1 [==============================] - 0s 252ms/step - loss: 1.0043
Validation: mean_average_precision(0.0): 0.5111336032388664
Epoch 4/5
1/1 [==============================] - 0s 68ms/step - loss: 1.0039
Validation: mean_average_precision(0.0): 0.5111336032388664
Epoch 5/5
1/1 [==============================] - 0s 247ms/step - loss: 1.0035
Validation: mean_average_precision(0.0): 0.5106275303643725
